In [ ]:
import os
import sys
HOME_DIR = '/Users/anhdang/Documents/Github/COS40007-Artificial-Intelligence-for-Engineering'
os.chdir(HOME_DIR)

DATASETS_DIR = f'{HOME_DIR}/datasets'
DATA_DIR = f'{DATASETS_DIR}/01'
DATASET_FILE = f'{DATA_DIR}/CCPP/Folds5x2_pp.ods'

print(f'HOME_DIR: {HOME_DIR}')
print(f'DATASETS_DIR: {DATASETS_DIR}')
print(f'DATA_DIR: {DATA_DIR}')
print(f'DATASET_FILE: {DATASET_FILE}')

In [ ]:
!pip install --upgrade pip --quiet
!pip install -r requirements.txt --quiet